In [35]:
# Load libraries and environment variables.

from selenium import webdriver
import time
from dotenv import load_dotenv
load_dotenv()
import os
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.edge.options import Options as EdgeOptions
from datetime import datetime

if load_dotenv():
    print("Variables de entorno cargadas.")
else:
    print("No se pudieron cargar las variables de entorno.")


Variables de entorno cargadas.


In [2]:
# Accede a las variables de entorno
USUARIO = os.getenv('USUARIO')
CONTRASEÑA = os.getenv('PASSWORD')

In [23]:
def calcular_posicion_calendario(dia, mes, anio):
    # Días en cada mes (asumiendo año no bisiesto)
    dias_por_mes = [31, 28, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31]
    
    # Ajuste para año bisiesto
    if anio % 4 == 0 and (anio % 100 != 0 or anio % 400 == 0):
        dias_por_mes[1] = 29
    
    # Calcular el día de la semana del 1 del mes
    a = (14 - mes) // 12
    y = anio - a
    m = mes + 12 * a - 2
    dia_semana_1_mes = (1 + y + y//4 - y//100 + y//400 + (31*m)//12) % 7
    
    # Ajustar para que el domingo sea 6 en lugar de 0
    dia_semana_1_mes = 6 if dia_semana_1_mes == 0 else dia_semana_1_mes - 1
    
    # Calcular la posición en el calendario de 41 posiciones
    posicion = dia_semana_1_mes + dia - 1
    
    # Ajustar si el día es del mes anterior
    if posicion < 0:
        mes_anterior = 12 if mes == 1 else mes - 1
        dias_mes_anterior = dias_por_mes[mes_anterior - 1]
        posicion = dias_mes_anterior + posicion
    
    return posicion

def obtener_info_calendario(mes, anio):
    dias_por_mes = [31, 28, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31]
    if anio % 4 == 0 and (anio % 100 != 0 or anio % 400 == 0):
        dias_por_mes[1] = 29
    
    primer_dia = calcular_posicion_calendario(1, mes, anio)
    dias_en_mes = dias_por_mes[mes - 1]
    ultima_posicion = primer_dia + dias_en_mes - 1
    
    return primer_dia, ultima_posicion, dias_en_mes

In [41]:
def hacer_reserva(USUARIO, CONTRASEÑA, dias, asiento):
    
    # Define the edge and driver options.
    edge_options = EdgeOptions()
    edge_options.use_chromium = True
    # edge_options.add_argument('--headless')
    # edge_options.add_argument('--no-sandbox')
    # edge_options.add_argument('--disable-dev-shm-usage')
    # edge_options.add_argument('--window-size=1920x1080')
    driver = webdriver.Edge(executable_path='msedgedriver.exe')
    
    # Navigate to the URL
    driver.get("https://allianz.swa.steelcase.com/")
    time.sleep(5)

    # Introduce el nombre de usuario y contraseña
    driver.find_element(By.ID, "username").send_keys(USUARIO)
    driver.find_element(By.ID, "password").send_keys(CONTRASEÑA)

    # Click en el botón de iniciar sesión
    boton_iniciar_sesion = driver.find_element(By.CSS_SELECTOR, 'button[data-action-button-primary="true"][value="default"]')
    boton_iniciar_sesion.click()
    time.sleep(10)
    
    for dia in dias:
        
        driver.find_element(By.LINK_TEXT, "Solicitar reserva de puesto de trabajo").click()
        time.sleep(10)
        driver.find_element(By.CSS_SELECTOR, ".title > .btn:nth-child(1)").click()
        time.sleep(3)

        # Reserva para el día especificado
        
        from datetime import datetime
        fecha = datetime.now()
        mes_actual = int(fecha.month)
        anio_actual = int(fecha.year)
        posicion = calcular_posicion_calendario(dia, mes_actual, anio_actual)
        driver.find_element(By.XPATH, f"//*[starts-with(@id, 'datepicker-') and contains(@id, '-{posicion}')]/button").click()
        

        driver.find_element(By.XPATH,"/html/body/div[1]/div/div/div[3]/button[2]").click()

        driver.find_element(By.ID,"Hasta").click()
        for i in range(0,6):
            driver.find_element(By.XPATH, "//*[@id='modal-body']/div/table/tbody/tr[1]/td[1]/a").click()

        time.sleep(10)
        driver.find_element(By.XPATH,"/html/body/div[1]/div/div/div[3]/button[2]").click()
        time.sleep(10)
        driver.find_element(By.XPATH,"/html/body/div[1]/div[2]/div[2]/div/div[2]/div[2]/div[1]/div[4]").click()
        time.sleep(15)
        # https://allianz.swa.steelcase.com/reservas/#!/ver-reserva/665823
        # driver.find_element(By.CSS_SELECTOR, ".leaflet-marker-icon:nth-child("+ str(asiento) + ")").click()
        driver.get("https://allianz.swa.steelcase.com/reservas/#!/realizar-reserva/" + str(asiento))
        time.sleep(10)
        driver.find_element(By.XPATH, "/html/body/div[1]/div[2]/div[2]/div/div[2]/div/div[2]/div[2]/ul[2]/li[2]/label/input").click()

        driver.find_element(By.XPATH, "/html/body/div[1]/div[2]/div[2]/div/div[2]/div/div[2]/div[2]/ul[2]/li[3]/button").click()
        time.sleep(3)

        driver.find_element(By.XPATH, "/html/body/div[1]/div/div/div[3]/button[2]").click()
        
        # Al finalizar las acciones para ese día, puedes esperar unos segundos antes de continuar con el siguiente día (opcional)
        time.sleep(5)
        
        driver.get("https://allianz.swa.steelcase.com/")
        
        time.sleep(15)


In [42]:
# Execute the script.

hacer_reserva(USUARIO, CONTRASEÑA, dias=[15], asiento=665823)